## Session Initialization

In [1]:
from pathlib import Path
import pyspark
from pyspark import SparkContext

sc = SparkContext()
spark = pyspark.sql.SparkSession(sc, jsparkSession=None)

## Join Data Frames

In [2]:
path_1 = Path.cwd().parent / "Files" / "bookcontents.csv"
path_2 = Path.cwd().parent / "Files" / "sections.csv"
book = str(path_1)
section = str(path_2)

In [3]:
# Load Chapters
bookChapterDF = spark.read.option("InferSchema", "True").option("header", "True").csv(book)

In [4]:
bookChapterDF.show()

+-------+--------------------+----+
|Chapter|                Name|Page|
+-------+--------------------+----+
|      1|        Introduction|  11|
|      2|Basic Engineering...|  19|
|      3|Advanced Engineer...|  28|
|      4|     Hands On Course|  60|
|      5|        Case Studies|  62|
|      6|Best Practices Cl...|  73|
|      7|130+ Data Sources...|  77|
|      8|1001 Interview Qu...|  82|
|      9|Recommended Books...|  87|
+-------+--------------------+----+



In [5]:
bookChapterDF.printSchema()

root
 |-- Chapter: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Page: integer (nullable = true)



In [6]:
# Load Sections
sectionDF = spark.read.option("InferSchema","True").option("header","True").csv(section)

In [7]:
sectionDF.show(9)

+-------+-------+--------------------+
|Chapter|Section|        Section_Name|
+-------+-------+--------------------+
|      1|    1.1|What is this Cook...|
|      1|    1.2|Data Engineer vs ...|
|      1|    1.3|My Data Science P...|
|      1|    1.4|  Who Companies Need|
|      2|    2.1|       Learn To Code|
|      2|    2.2|Get Familiar With...|
|      2|    2.3|   Agile Development|
|      2|    2.4|Software Engineer...|
|      2|    2.5|Learn how a Compu...|
+-------+-------+--------------------+
only showing top 9 rows



In [8]:
sectionDF.printSchema()

root
 |-- Chapter: integer (nullable = true)
 |-- Section: double (nullable = true)
 |-- Section_Name: string (nullable = true)



In [9]:
sectionDF.where("Chapter == 5").show(10)

+-------+-------+--------------------+
|Chapter|Section|        Section_Name|
+-------+-------+--------------------+
|      5|    5.1| Data Science Airbnb|
|      5|    5.2| Data Science Amazon|
|      5|    5.3|  Data Science Baidu|
|      5|    5.4|Data Science Blac...|
|      5|    5.5|    Data Science BMW|
|      5|    5.6|Data Science Book...|
|      5|    5.7|   Data Science CERN|
|      5|    5.8| Data Science Disney|
|      5|    5.9|    Data Science DLR|
|      5|    5.1|Data Science Driv...|
+-------+-------+--------------------+
only showing top 10 rows



In [10]:
# join
# inner,outer,left_outer,right_outer,leftsemi
bookChapterDF.join(sectionDF, "Chapter", "inner").where("Chapter == 1 or Chapter == 2").show()

+-------+--------------------+----+-------+--------------------+
|Chapter|                Name|Page|Section|        Section_Name|
+-------+--------------------+----+-------+--------------------+
|      1|        Introduction|  11|    1.1|What is this Cook...|
|      1|        Introduction|  11|    1.2|Data Engineer vs ...|
|      1|        Introduction|  11|    1.3|My Data Science P...|
|      1|        Introduction|  11|    1.4|  Who Companies Need|
|      2|Basic Engineering...|  19|    2.1|       Learn To Code|
|      2|Basic Engineering...|  19|    2.2|Get Familiar With...|
|      2|Basic Engineering...|  19|    2.3|   Agile Development|
|      2|Basic Engineering...|  19|    2.4|Software Engineer...|
|      2|Basic Engineering...|  19|    2.5|Learn how a Compu...|
|      2|Basic Engineering...|  19|    2.6|Data Network Tran...|
|      2|Basic Engineering...|  19|    2.7|Security and Privacy|
|      2|Basic Engineering...|  19|    2.8|               Linux|
|      2|Basic Engineerin

## Access Columns

In [11]:
bookContentDF = bookChapterDF.join(sectionDF, "Chapter")
bookContentDF.show(9)

+-------+--------------------+----+-------+--------------------+
|Chapter|                Name|Page|Section|        Section_Name|
+-------+--------------------+----+-------+--------------------+
|      1|        Introduction|  11|    1.1|What is this Cook...|
|      1|        Introduction|  11|    1.2|Data Engineer vs ...|
|      1|        Introduction|  11|    1.3|My Data Science P...|
|      1|        Introduction|  11|    1.4|  Who Companies Need|
|      2|Basic Engineering...|  19|    2.1|       Learn To Code|
|      2|Basic Engineering...|  19|    2.2|Get Familiar With...|
|      2|Basic Engineering...|  19|    2.3|   Agile Development|
|      2|Basic Engineering...|  19|    2.4|Software Engineer...|
|      2|Basic Engineering...|  19|    2.5|Learn how a Compu...|
+-------+--------------------+----+-------+--------------------+
only showing top 9 rows



In [12]:
bookContentDF.select(bookContentDF.Section_Name, "Chapter").show(9)

+--------------------+-------+
|        Section_Name|Chapter|
+--------------------+-------+
|What is this Cook...|      1|
|Data Engineer vs ...|      1|
|My Data Science P...|      1|
|  Who Companies Need|      1|
|       Learn To Code|      2|
|Get Familiar With...|      2|
|   Agile Development|      2|
|Software Engineer...|      2|
|Learn how a Compu...|      2|
+--------------------+-------+
only showing top 9 rows



## Working with Columns Expressions

In [13]:
# use aliases for columns
bookContentDF.select("Section_Name", (bookContentDF.Chapter * 10).alias("Chapter_10")).show()

+--------------------+----------+
|        Section_Name|Chapter_10|
+--------------------+----------+
|What is this Cook...|        10|
|Data Engineer vs ...|        10|
|My Data Science P...|        10|
|  Who Companies Need|        10|
|       Learn To Code|        20|
|Get Familiar With...|        20|
|   Agile Development|        20|
|Software Engineer...|        20|
|Learn how a Compu...|        20|
|Data Network Tran...|        20|
|Security and Privacy|        20|
|               Linux|        20|
|              Docker|        20|
|           The Cloud|        20|
|Security Zone Design|        20|
|Data Science Plat...|        30|
|    Hadoop Platforms|        30|
|             Connect|        30|
|              Buffer|        30|
|Processing Framew...|        30|
+--------------------+----------+
only showing top 20 rows



In [14]:
# Select something specific
bookContentDF.where(bookContentDF.Name.contains("Case") & bookContentDF.Section_Name.contains("Data")).show()

+-------+------------+----+-------+--------------------+
|Chapter|        Name|Page|Section|        Section_Name|
+-------+------------+----+-------+--------------------+
|      5|Case Studies|  62|    5.1| Data Science Airbnb|
|      5|Case Studies|  62|    5.2| Data Science Amazon|
|      5|Case Studies|  62|    5.3|  Data Science Baidu|
|      5|Case Studies|  62|    5.4|Data Science Blac...|
|      5|Case Studies|  62|    5.5|    Data Science BMW|
|      5|Case Studies|  62|    5.6|Data Science Book...|
|      5|Case Studies|  62|    5.7|   Data Science CERN|
|      5|Case Studies|  62|    5.8| Data Science Disney|
|      5|Case Studies|  62|    5.9|    Data Science DLR|
|      5|Case Studies|  62|    5.1|Data Science Driv...|
|      5|Case Studies|  62|   5.11|Data Science Dropbox|
|      5|Case Studies|  62|   5.12|   Data Science Ebay|
|      5|Case Studies|  62|   5.13|Data Science Expedia|
|      5|Case Studies|  62|   5.14|Data Science Face...|
|      5|Case Studies|  62|   5

## Group By & Aggregation Function

In [15]:
# Group by aggregation options:
# agg(additional functions), count, mean, max or min, pivot, sum
bookContentDF.groupBy(bookContentDF.Chapter).count().show()

+-------+-----+
|Chapter|count|
+-------+-----+
|      1|    4|
|      6|    3|
|      3|    8|
|      5|   37|
|      9|    3|
|      4|    7|
|      8|    1|
|      7|   19|
|      2|   11|
+-------+-----+

